In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_t-112,rides_t-111,rides_t-110,rides_t-109,rides_t-108,rides_t-107,rides_t-106,rides_t-105,rides_t-104,rides_t-103,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,4,49,42,114,57,7,110,112,86,14,...,39,66,205,111,26,30,56,2024-01-28 18:00:00,5626,70
1,49,42,114,57,7,110,112,86,14,103,...,66,205,111,26,30,56,70,2024-01-29 00:00:00,5626,12
2,42,114,57,7,110,112,86,14,103,135,...,205,111,26,30,56,70,12,2024-01-29 06:00:00,5626,112
3,114,57,7,110,112,86,14,103,135,97,...,111,26,30,56,70,12,112,2024-01-29 12:00:00,5626,123
4,57,7,110,112,86,14,103,135,97,10,...,26,30,56,70,12,112,123,2024-01-29 18:00:00,5626,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6760,108,11,160,180,115,15,162,192,111,9,...,15,37,131,69,10,63,171,2024-12-30 18:00:00,6072,80
6761,11,160,180,115,15,162,192,111,9,139,...,37,131,69,10,63,171,80,2024-12-31 00:00:00,6072,15
6762,160,180,115,15,162,192,111,9,139,162,...,131,69,10,63,171,80,15,2024-12-31 06:00:00,6072,87
6763,180,115,15,162,192,111,9,139,162,120,...,69,10,63,171,80,15,87,2024-12-31 12:00:00,6072,142


In [3]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 10, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4925, 114)
(4925,)
(1840, 114)
(1840,)


In [4]:
import numpy as np

class BaselineModelPreviousHour:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]

In [5]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)

In [6]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

139.0484


In [7]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 
import os
#os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/api/v1/repo-buckets/s3/thanoojlingampally"
mlflow = set_mlflow_tracking()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as thanoojlingampally

INFO:dagshub:Accessing as thanoojlingampally
INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/thanoojlingampally/citi_bike "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "thanoojlingampally/citi_bike"

INFO:dagshub:Initialized MLflow to track repo "thanoojlingampally/citi_bike"


Repository thanoojlingampally/citi_bike initialized!

INFO:dagshub:Repository thanoojlingampally/citi_bike initialized!
INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [8]:
%pip install -q dagshub mlflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import dagshub
import mlflow
dagshub.init(repo_owner='thanoojlingampally', repo_name='citi_bike', mlflow=True)

# Set the MLflow Tracking URI via Dagshub initialization
print("✅ MLflow Tracking URI:", mlflow.get_tracking_uri())

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/thanoojlingampally/citi_bike "HTTP/1.1 200 OK"


Initialized MLflow to track repo "thanoojlingampally/citi_bike"

INFO:dagshub:Initialized MLflow to track repo "thanoojlingampally/citi_bike"


Repository thanoojlingampally/citi_bike initialized!

INFO:dagshub:Repository thanoojlingampally/citi_bike initialized!


✅ MLflow Tracking URI: https://dagshub.com/thanoojlingampally/citi_bike.mlflow


In [10]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousHour", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 139.0483695652174
c:\Users\thano\anaconda3\envs\cda\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.


Registered model 'BaselineModelPreviousHour' already exists. Creating a new version of this model...
2025/05/11 03:30:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousHour, version 3
Created version '3' of model 'BaselineModelPreviousHour'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousHour


🏃 View run righteous-kite-838 at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/0/runs/f5cac4102be54a3d8b56e7e671534d1e
🧪 View experiment at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/0


In [13]:
import numpy as np

class BaselineModelPreviousWeek:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*4}"]

In [14]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)

In [15]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

46.5973


In [16]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 46.59728260869565
c:\Users\thano\anaconda3\envs\cda\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.


Registered model 'BaselineModelPreviousWeek' already exists. Creating a new version of this model...
2025/05/11 03:34:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousWeek, version 2
Created version '2' of model 'BaselineModelPreviousWeek'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousWeek


🏃 View run kindly-fish-320 at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/1/runs/d05ebcd541894fc2ac936e56a310ba2f
🧪 View experiment at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/1


In [21]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        The fit method is not used in this baseline model as it does not learn
        from the training data.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predicts the average of the last 4 weeks (28 days) for each test instance.

        Parameters:
            X_test (pd.DataFrame): The test DataFrame containing lagged features
                                   (e.g., rides_t-{7*24}, rides_t-{14*24}, etc.).

        Returns:
            np.array: An array of predictions based on the average of the last 4 weeks.
        """
        # Define the columns for the last 4 weeks
        last_4_weeks_columns = [
            f"rides_t-{7*4}",  # 1 week ago
            f"rides_t-{14*4}", # 2 weeks ago
            f"rides_t-{21*4}", # 3 weeks ago
            f"rides_t-{28*4}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")

        # Calculate the average of the last 4 weeks
        predictions = X_test[last_4_weeks_columns].mean(axis=1)

        return predictions.to_numpy()

In [22]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)

In [23]:
X_test[X_test['pickup_location_id']==5626] 

,rides_t-112,rides_t-111,rides_t-110,rides_t-109,rides_t-108,rides_t-107,rides_t-106,rides_t-105,rides_t-104,rides_t-103,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,39,331,298,302,51,328,298,334,43,340,...,101,82,209,217,45,286,335,240,2024-10-01 00:00:00,5626
1,331,298,302,51,328,298,334,43,340,290,...,82,209,217,45,286,335,240,33,2024-10-01 06:00:00,5626
2,298,302,51,328,298,334,43,340,290,319,...,209,217,45,286,335,240,33,330,2024-10-01 12:00:00,5626
3,302,51,328,298,334,43,340,290,319,61,...,217,45,286,335,240,33,330,310,2024-10-01 18:00:00,5626
4,51,328,298,334,43,340,290,319,61,286,...,45,286,335,240,33,330,310,310,2024-10-02 00:00:00,5626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,131,23,207,218,179,24,232,203,150,12,...,60,36,78,159,107,19,111,215,2024-12-30 18:00:00,5626
364,23,207,218,179,24,232,203,150,12,240,...,36,78,159,107,19,111,215,144,2024-12-31 00:00:00,5626
365,207,218,179,24,232,203,150,12,240,205,...,78,159,107,19,111,215,144,40,2024-12-31 06:00:00,5626
366,218,179,24,232,203,150,12,240,205,187,...,159,107,19,111,215,144,40,95,2024-12-31 12:00:00,5626


In [24]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

45.3031


In [25]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)

INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 45.303125
c:\Users\thano\anaconda3\envs\cda\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.


Registered model 'BaselineModelLast4Weeks' already exists. Creating a new version of this model...
2025/05/11 03:36:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelLast4Weeks, version 2
Created version '2' of model 'BaselineModelLast4Weeks'.
INFO:src.experiment_utils:Model logged with name: BaselineModelLast4Weeks


🏃 View run popular-kite-909 at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/2/runs/99a8929fc1f14879b800e010212d35b7
🧪 View experiment at: https://dagshub.com/thanoojlingampally/citi_bike.mlflow/#/experiments/2


In [27]:
from src.plot_utils import plot_aggregated_time_series_6h

plot_aggregated_time_series_6h(X_test, y_test, 200, predictions)